# My Reaction - Advection - Diffusion Model

## 3 species: vegetation, combustible gases, heat

### Reaction 1: Vegetation + Heat -> Combustible Gas
### Reaction 2: Heat + Combustible Gas -> Heat

# System of PDEs
## (1) $\frac{\partial u_1}{\partial t} = -K_1 u_1 u_3$
## (2) $\frac{\partial u_2}{\partial t} + w \cdot \nabla u_2 - \nabla \cdot (\epsilon \nabla u_2) = K_1 u_1 u_3 - K_2 u_2 u_3$
## (3) $\frac{\partial u_3}{\partial t} + w \cdot \nabla u_3 - \nabla \cdot (\epsilon \nabla u_3) = K_2 u_2 u_3 - K_1 u_1 u_3$

# Variational Form
## (1) $\int_{\Omega} \frac{\partial u_1}{\partial t} v_1 - \int_{\Omega} -K_1 u_1 u_3 v_1 = 0$
## (2) $\int_{\Omega} \frac{\partial u_2}{\partial t} v_2 + w \cdot \nabla u_2 v_2 - \epsilon \nabla u_2 \nabla v_2 - \int_{\Omega} (K_1 u_1 u_3 - K_2 u_2 u_3) v_2 = 0$
## (3) $\int_{\Omega} \frac{\partial u_3}{\partial t} v_3 + w \cdot \nabla u_3 v_3 - \epsilon \nabla u_3 \nabla v_3 - \int_{\Omega} (K_2 u_2 u_3 - K_1 u_1 u_3) v_3 = 0$

In [12]:
from firedrake import *
from tqdm import tqdm
from scipy import constants
import numpy as np
import matplotlib.pyplot as plt

In [13]:
# Mesh parameters
Lx = 5
Ly = 4
Nx = 50
Ny = 40
mesh = RectangleMesh(Nx, Ny, Lx, Ly)
x, y = SpatialCoordinate(mesh)

In [14]:
# Simulation time parameters
T = 5.
num_steps = 50
dt_step = T / num_steps
dt = Constant(dt_step)
t = 0

In [15]:
# Model parameters
eps_heat = Constant(0.5)
eps_gas = Constant(0.01)  # Diffusion Rate
K = Constant(1)


In [16]:
# Advection Term
w_uv = as_vector((
    0.1,
    0
))
W = VectorFunctionSpace(mesh, family='CG', degree=2, dim=2)
w = interpolate(w_uv, W)

In [17]:
# Plot the wind field
# fig, axes = plt.subplots(figsize=(12, 12))
# axes.set_title('Advection Field')
# opts = {'resolution': 1/32, 'seed': 1}
# streamlines = streamplot(w, axes=axes, **opts)
# fig.colorbar(streamlines)

In [18]:
# Define the mixed elements
P1 = FiniteElement('P', triangle, 1)
element = MixedElement([P1, P1, P1])

# Define the test functions
V = FunctionSpace(mesh, element)
v_1, v_2, v_3 = TestFunctions(V)

# No trial functions since the problem is nonlinear
u = Function(V)
u_n = Function(V)


In [19]:
# Define Fuel initial conditions
u.sub(0).assign(project(Constant(1), V[0]))
u_n.sub(0).assign(project(Constant(1), V[0]))

# Define Heat Initial Condition
heat_ic = conditional(pow(x - 0.75, 2) + pow(y - 2, 2) < 0.25**2, 10, 0)
u.sub(2).assign(project(heat_ic, V[2]))
u_n.sub(2).assign(project(heat_ic, V[2]))

Coefficient(WithGeometry(IndexedProxyFunctionSpace(<firedrake.mesh.MeshTopology object at 0x137fbf0a0>, FiniteElement('Lagrange', triangle, 1), name=None, index=2, component=None), Mesh(VectorElement(FiniteElement('Lagrange', triangle, 1), dim=2), 328)), 686)

In [20]:
# Pull variables from u and u_n function spaces
u_1, u_2, u_3 = split(u)
u_n1, u_n2, u_n3 = split(u_n)

In [21]:
# Build the variational form of the elements
fuel_element = (((u_1 - u_n1) / dt)*v_1)*dx
gas_element = ((u_2 - u_n2) / dt)*v_2*dx + dot(w, grad(u_2))*v_2*dx + eps_gas*dot(grad(u_2), grad(v_2))*dx
heat_element = ((u_3 - u_n3) / dt)*v_3*dx + dot(w, grad(u_3))*v_3*dx + eps_heat*dot(grad(u_3), grad(v_3))*dx
elements_variational_form = fuel_element + gas_element + heat_element

# Build each reaction
# K_arr = A * exp(E_A/(R * u_3))
# K_arr = conditional(ge(u_3, u_pc), 100, 0.1)
fuel_heat_reaction = K * u_1 * u_3
gas_heat_reaction = K * u_2 * u_3

# Build the variational form of the reactions
fuel_reactions = -fuel_heat_reaction * v_1 * dx
gas_reactions = (fuel_heat_reaction - gas_heat_reaction) * v_2 * dx
heat_reactions = (10*gas_heat_reaction - fuel_heat_reaction) * v_3 * dx
reactions_variational_form = fuel_reactions + gas_reactions + heat_reactions

# Combine the variational forms
F = elements_variational_form - reactions_variational_form


In [22]:
problem = NonlinearVariationalProblem(F, u)
solver = NonlinearVariationalSolver(problem)

# Initialize time
t=0

# Create VTK files for visualization output
vtkfile_u_1 = File('my_model_output/fuel.pvd')
vtkfile_u_2 = File('my_model_output/gas.pvd')
vtkfile_u_3 = File('my_model_output/heat.pvd')
_u_1, _u_2, _u_3 = u.split()
vtkfile_u_1.write(_u_1, time=t)
vtkfile_u_2.write(_u_2, time=t)
vtkfile_u_3.write(_u_3, time=t)

# Solve the system
for n in tqdm(range(num_steps), leave=True, position=0):
    t += dt_step

    # w_uv = as_vector((
    #     2,
    #     sin(t**2)
    # ))
    # w = interpolate(w_uv, W)

    # solve(F == 0, u)
    solver.solve()

    # Save solution to file (VTK)
    _u_1, _u_2, _u_3 = u.split()
    vtkfile_u_1.write(_u_1, time=t)
    vtkfile_u_2.write(_u_2, time=t)
    vtkfile_u_3.write(_u_3, time=t)

    # Update previous solution
    u_n.assign(u)

100%|██████████| 50/50 [00:11<00:00,  4.39it/s]
